<a href="https://colab.research.google.com/github/fasfaisa/emotion_detection/blob/main/face_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
# Deep Learning Libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from google.colab import drive
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [8]:


# Mount Google Drive
drive.mount('/content/drive')

def load_images_from_folder(folder_path):
    """
    Load images from a folder with subdirectories representing emotions

    Args:
        folder_path (str): Path to the main folder containing emotion subfolders

    Returns:
        tuple: (images, labels, emotion_map)
    """
    images = []
    labels = []
    emotion_map = {}

    # Define emotions based on subdirectory names
    emotions = sorted(os.listdir(folder_path))

    # Create emotion to index mapping
    for idx, emotion in enumerate(emotions):
        emotion_map[emotion] = idx

        # Path to specific emotion folder
        emotion_path = os.path.join(folder_path, emotion)

        # Skip if not a directory
        if not os.path.isdir(emotion_path):
            continue

        # Load images for this emotion
        for filename in os.listdir(emotion_path):
            img_path = os.path.join(emotion_path, filename)

            try:
                # Read image
                img = load_img(img_path, target_size=(48, 48), color_mode='grayscale')

                # Convert to numpy array
                img_array = img_to_array(img)

                # Normalize
                img_array /= 255.0

                images.append(img_array)
                labels.append(idx)

            except Exception as e:
                print(f"Error loading {img_path}: {e}")

    return (np.array(images), np.array(labels), emotion_map)

# Set paths
zip_path = '/content/drive/MyDrive/archive.zip'
extract_path = '/content/drive/MyDrive/FER2013/extracted'

# Load images
X, y, emotion_labels = load_images_from_folder(os.path.join(extract_path, 'train'))

# Reshape for CNN
X = X.reshape(-1, 48, 48, 1)

# One-hot encode labels
y = to_categorical(y, num_classes=len(emotion_labels))

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Mounted at /content/drive


In [9]:
# Step 4:

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    validation_split=0.2
)

datagen.fit(X_train)

In [10]:

# Advanced CNN Model
def create_emotion_model(input_shape=(48, 48, 1), num_classes=len(emotion_labels)):
    model = Sequential([
        # First Convolutional Block
        Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        # Second Convolutional Block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        # Third Convolutional Block
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        # Flatten and Dense Layers
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [11]:

# Create and compile the model
model = create_emotion_model()

# Ensure .keras extension for model checkpoint
checkpoint_path = '/content/drive/MyDrive/best_emotion_model.keras'

# Callbacks
model_checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

lr_reducer = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=0.00001
)

early_stopper = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:

# Step 7: Train the Model
# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=64, subset='training'),
    validation_data=datagen.flow(X_train, y_train, subset='validation'),
    epochs=50,
    callbacks=[model_checkpoint, lr_reducer, early_stopper]
)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 10/288 ━━━━━━━━━━━━━━━━━━━━ 22:15 5s/step - accuracy: 0.1555 - loss: 3.1330

In [ ]:

# Step 8: Model Evaluation
# Load the best saved model
best_model = load_model(checkpoint_path)

# Evaluate the model
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")

# Predictions
y_pred = best_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Classification Report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=emotion_labels))

# Confusion Matrix Visualization
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_true_classes, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=emotion_labels,
            yticklabels=emotion_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

In [ ]:

# Step 9: Training History Visualization
plt.figure(figsize=(12, 4))

# Plot Training Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot Training Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:

# Step 10: Real-time Emotion Detection Function
def detect_emotion(frame, face_cascade, emotion_model):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Load pre-trained Haar cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # Extract face ROI
        roi_gray = gray[y:y+h, x:x+w]

        # Resize to 48x48
        roi_gray = cv2.resize(roi_gray, (48, 48))

        # Normalize
        roi_gray = roi_gray / 255.0

        # Reshape for model input
        roi_gray = roi_gray.reshape(1, 48, 48, 1)

        # Predict emotion
        prediction = emotion_model.predict(roi_gray)
        emotion_index = np.argmax(prediction)

        # Draw rectangle and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion_labels[emotion_index], (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    return frame

# Save the model for future use
model.save('/content/drive/MyDrive/FER2013/emotion_recognition_model.h5')

print("Emotion Recognition Model Training Completed!")